# Spectral analysis

This notebook introduces spectral analysis in simple cases (noiseless signals, one or two sinusoids).

The Discrete Fourier Transform (computed using a Fast Fourier Transform algorithm) is used to estimate the frequency, amplitude, and phase of a sinusoid.

The following issues will be addressed:
- the discretization of the DTFT and its refinement with zero-padding
- separation of sinusoids with similar frequencies using a longer observation window
- mitigation of the bias introduced by side-lobes using appropriate windowing.


In [ ]:

# Imports and setup

%matplotlib notebook

import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import numpy.random as rnd
import scipy.fft as fft

# size of the figures
width = 8
height = 4

## Case of a unique complex exponential


### Data generation 

We simulate $L$ samples of a complex exponential of amplitude $A$, phase $\phi$ and frequency $\nu_0$ :
$$x[n] = A e^{i\phi} \exp(i 2\pi \nu_0 n)$$
for $0 \leq n < L$.

For modelization purposes, we extend $x[n]$ by setting $x[n] = 0$ for $n$ outside of the support.



In [ ]:
# length and time vector
L = 16
t = np.arange(L)

# parameters
nu_0 = 0.2241 # frequency
A = 2     # amplitude
phi = 1.2 # phase

e_nu = np.exp( 1j * 2 * np.pi * nu_0 * t) # complex sinusoid

x = A * np.exp(1j * phi) * e_nu

plt.figure(figsize=(width,height))

plt.stem(t, np.real(x), use_line_collection  = True)

# continuous time signal
tcont = np.arange(L, step=0.01)
plt.plot(tcont, np.real(A * np.exp(1j * phi) * np.exp( 1j * 2 * np.pi * nu_0 * tcont)), '-g')


plt.xlabel('Time (samples)')
plt.ylabel('Re(x[n])');
plt.legend(['Continuous sinusoid','Samples x[n]']);

### DTFT and DFT

The DTFT $X(\nu)$ of the signal $x[n]$, defined by
$$X(\nu) = \sum_{n\in\mathbf Z} x[n] \exp(-i2\pi \nu n)$$
is the circular convolution of a Dirac distribution at frequency $\nu_0$ and the DTFT of the square window, multiplied by $Ae^{i\phi}$.
It has a maximum at the actual frequency $\nu_0$.

We plot the DFT $\mathbf X$ of the signal vector $\mathbf x$, a vector of size $L$ such that $ x_n = x[n]$ for $0 \leq n < L$. The values of the DFT, given by
$$X_k = \sum_{n = 0}^{L-1} x_n \exp(- i 2\pi nk /L)$$
are samples of the DTFT of $x[n]$:
$$X_k = X(k/L).$$


The parameters are identified by searching for the maximal value of the magnitude of the DFT.
 
**Q1** Explain the mismatch between the estimated values and the actual values of the parameters.




In [ ]:


nu_fft = np.arange(L)/L - 1/2 # frequencies of the FFT values (/!\ only for even L)

F = fft.fft(x) # Computation of the FFT

Fshift = fft.fftshift(F) # shifting from [0,1) to [-1/2, 1/2)

plt.figure(figsize=(width,height))
plt.semilogy(nu_fft, ( np.abs(Fshift)**2 / L**2), '-+')
plt.xlabel('Frequency')
plt.ylabel('Power');
plt.legend(['|DFT of x|^2']),

# estimated parameters
idx_est = np.argmax(np.abs(Fshift))

nu_est = nu_fft[idx_est]
A_est = np.abs(F[idx_est]) / L
phi_est = np.angle(F[idx_est])


print("Estimated frequency : {:.3f} (actual {:.3f})".format(nu_est, nu_0))
print("Estimated amplitude : {:.3f} (actual {:.3f})".format(A_est, A))
print("Estimated phase : {:.3f} (actual {:.3f})".format(phi_est, phi))

### Zero-padding

We use zero-padding to improve the estimation of the parameters: the vector $\mathbf x$ is extended to a length $N$ with zero values.

**Q2** Explain how zero-padding improves the estimation. Implement zero-padding using the second argument of fft.



In [ ]:


# Modify from here...
nu_pad = np.arange(L)/L -1/2 # frequencies of the FFT values
F_pad = fft.fft(x) # Computation of the DTFT
F_pad_shift = fft.fftshift(F_pad)
# ...to here



plt.figure(figsize=(width,height))
plt.semilogy(nu_pad, np.abs(F_pad_shift)**2/L**2, '-+')
plt.semilogy(nu_fft, np.abs(Fshift)**2/L**2, '-+')
plt.xlabel('Frequency')
plt.ylabel('Power');
plt.legend(['With 0-padding', 'Without 0-padding']),


# estimated parameters
idx_est = np.argmax(np.abs(F_pad_shift))


nu_est = nu_pad[idx_est]
A_est = np.abs(F_pad_shift[idx_est]) / L
phi_est = np.angle(F_pad_shift[idx_est])


print("Estimated frequency : {:.3f} (actual {:.3f})".format(nu_est, nu_0))
print("Estimated amplitude : {:.3f} (actual {:.3f})".format(A_est, A))
print("Estimated phase : {:.3f} (actual {:.3f})".format(phi_est, phi))


## Case of two sinusoids

In the case of $K$ sinusoids, $K > 1$, estimation of the parameters can be difficult for two main reasons :
- sidelobes, which can bias the estimation of the amplitude of a sinusoid, or even hide it
- nonzero width of the mainlobe, which can prevent the identification of closely spaced sinusoids.


In [ ]:
# length and time vector
L2 = 24
t = np.arange(L2)

# parameters
nu1 = 0.12 # frequency
A1 = 2     # amplitude
phi1 = 1.2 # phase

nu2 = 0.22 # frequency
A2 = 2    # amplitude
phi2 = -1 # phase

e_nu1 = np.exp( 1j * 2 * np.pi * nu1 * t) # noiseless sinusoid
e_nu2 = np.exp( 1j * 2 * np.pi * nu2 * t) # noiseless sinusoid

x2 = A1 * np.exp(1j * phi1) * e_nu1 + A2 * np.exp(1j * phi2) * e_nu2

plt.figure(figsize=(width,height))

plt.stem(t, np.real(x2));


# continuous time version of the noiseless signal
tcont = np.arange(L2, step=0.01)
plt.plot(tcont, np.real(A1 * np.exp(1j * phi1) * np.exp( 1j * 2 * np.pi * nu1 * tcont) + A2 * np.exp(1j * phi2) * np.exp( 1j * 2 * np.pi * nu2 * tcont)), '--')



plt.xlabel('Time (samples)')
plt.ylabel('x[n]');
plt.legend(['Samples', 'Continuous signal']);

### Resolution

The spectrum is plotted using $L_{short} = 8$ samples. The two sinusoids cannot be resolved.

**Q3** Suggest a longer window length $L_{long}$ allowing resolution of the sinusoids.




In [ ]:
Nfft = 2000 # number of FFT points

nu_fft = np.arange(Nfft)/Nfft - 1/2 # frequencies of the FFT values
F2 = fft.fft(x2, Nfft) # Computation of the DTFT


plt.figure(figsize=(width,height))

# Short window
Lshort = 8
F_short = fft.fftshift( fft.fft(x2[1:Lshort], Nfft)) # Computation of the DTFT
plt.semilogy(nu_fft, np.abs(F_short)**2/Lshort**2)

#Long window

# Modify from here...
Llong = 8
# ...to here


F_long = fft.fftshift( fft.fft(x2[1:Llong], Nfft)) # Computation of the DTFT
plt.semilogy(nu_fft, np.abs(F_long)**2/Llong**2)


# actual frequencies
plt.axvline(nu1, ls='--');
plt.axvline(nu2, ls='--');


plt.xlabel('Frequency')
plt.ylabel('Power');

plt.legend(['{:} samples'.format(Lshort),'{:} samples'.format(Llong)]);

### Windowing

Even if the sinusoids can be resolved, sidelobes of a sinusoid can bias the estimation of the amplitude of another, less powerful, sinusoid, or even hide it. Windowing the signal before computing the DTFT can improve the estimation of the amplitude of multiple sinusoids. 

**Q4** Plot the absolute value squared of DTFT of the Rectangular, Hamming, and Hann windows, normalized so that the value at the zero frequency is 1.

Compare
- the width of the mainlobes,
- the maximal height of the sidelobes,
- the decay of the sidelobes.




In [ ]:
import scipy.signal.windows as win
L_w = 40

w_square   = np.ones([L_w])
w_hann     =  win.hann(L_w)
w_hamming  =  win.hamming(L_w)



plt.figure(figsize=(width,height))

plt.plot(w_square,  '-+')
plt.plot(w_hann, '-+')
plt.plot(w_hamming, '-+')
plt.legend(['Square window', 'Hann window', 'Hamming window']);


plt.figure(figsize=(width,height))
#
#

In this example, two sinusoids are present in the signal. With no windowing, the frequency and amplitude
of the less powerful sinusoid are biased. Windowing the signal improves the estimation of its parameters.

**Q5** Use an appropriate window to allow accurate estimation of both sinusoids.



In [ ]:
L_sig = 40
nu_1 = 0.12
nu_2 = 0.43
sig = np.exp(1j * 2 * np.pi * nu_1 * np.arange(L_sig)) - 0.06 * np.exp(1j * 2 * np.pi * nu_2 * np.arange(L_sig))
 
Lpad = 1000
sig_fft_pad = fft.fftshift( fft.fft(sig, Lpad) )
nu_pad = np.arange(Lpad)/Lpad - 1/2
plt.figure(figsize=(width,height))


# Modify from here...
w = np.ones([L_sig])
# ...to here




W = np.sum(w)

plt.subplot(2, 1, 1)
plt.stem(np.arange(L_sig), np.real(sig))

plt.stem(np.arange(L_sig), np.real(sig) * w, linefmt='C1-', markerfmt='C1o')


sig_fft_w = fft.fftshift( fft.fft(sig*w, Lpad) )
plt.subplot(2, 1, 2)

plt.semilogy(nu_pad, (np.abs(sig_fft_pad)/ L_sig)**2);

plt.semilogy(nu_pad, np.abs(sig_fft_w)**2/ W**2);

plt.axvline(nu_1, ls='--')
plt.axvline(nu_2, ls='--')

plt.legend(['Square window', '? window']);

plt.xlabel('Frequency')
plt.ylabel('Power');